In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#导入其他必要的库
from  sklearn.ensemble import RandomForestRegressor

1.查看数据

In [ ]:
#查看训练集
train=pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-training.csv", index_col=False)
train.head(10)

#Unnamed 序号

#
#RevolvingUtiilizationOfUnsecuredLines （个人信用卡额度+个人信用额度）/个人信用总额度
#NumberOfOpenCreditLinesAndLoans 未偿还贷款笔数（诸如车贷或抵押贷款等分期付款）和个人信用贷（如信用卡）
#NumberRealEstateLoansOrLines 抵偿贷款和房地产贷款数量，包括房屋净值信贷额度等

#个人信息
#Age 借贷人年龄
#NumberOfDependents 借贷人家庭人数

#还债能力
#DebtRatio 负债率=（每月偿还债务+赡养费+生活费用）/每月总收入
#Monthlylncome 月收入

#信誉情况
#SeriousDlqin2yrs 2年内逾期天数是否超过90天（好用户和坏用户）
#NumberOfTime30-59DaysPastDueNotWorse 逾期30-59天的次数
#NumberOfTime60-89DaysPastDueNotWorse 逾期60-89天的次数
#NumberOfTimes90DaysLate 逾期90天以上的次数



In [ ]:
#查看测试集
test=pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-test.csv", index_col=False)
test.head()

In [ ]:
trainunnamed = train['Unnamed: 0']
Unnamed = test['Unnamed: 0']

2.清洗数据

2.1缺失值处理

In [ ]:
#剔除无用行
del train['Unnamed: 0']
train.info()
#月收入和家庭人数有缺失

In [ ]:
del test['Unnamed: 0']
test.info()

In [ ]:
#随机森林处理缺失值
#随机森林法填补MonthlyIncome
MI_train=train.loc[:,'SeriousDlqin2yrs':'NumberOfTime60-89DaysPastDueNotWorse']
MI_known=MI_train[MI_train['MonthlyIncome'].notnull()]
MI_unknown=MI_train[MI_train['MonthlyIncome'].isnull()]

# 删除MI_know中的MonthlyIncome列，但不改变原有的MI_know中的数据，而是返回另一个dataframe：X_known来存放删除后的数据
X_known=MI_known.drop('MonthlyIncome',axis=1)
# y_known是一个series
y_known=MI_known['MonthlyIncome']
# 删除MI_unknown中的MonthlyIncome列，但不改变原有的MI_unknown中的数据，而是返回另一个dataframe：X_unknown来存放删除后的数据
X_unknown=MI_unknown.drop('MonthlyIncome',axis=1)

# 使用scikit-learn中的RandomForestRegressor
# """
# random_state：
# 此参数让结果容易复现。 一个确定的随机值将会产生相同的结果，在参数和训练数据不变的情况下。
# 有时候这种方法比单独的随机状态更好。
# n_estimators=100:决策树的个数，越多越好，但是越多性能就会越差，至少100左右
# max_depth: (default=None)设置树的最大深度，默认为None，这样建树时，会使每一个叶节点只有一个类别，或是达到min_samples_split。
# n_jobs=1：并行job个数。这个在ensemble算法中非常重要，尤其是bagging
# （而非boosting，因为boosting的每次迭代之间有影响，所以很难进行并行化），因为可以并行从而提高性能。
# 1=不并行；n：n个并行；-1：CPU有多少core，就启动多少job
# """
rfr=RandomForestRegressor(random_state=0, n_estimators=100, max_depth=3, n_jobs=-1)
rfr.fit(X_known, y_known)
train.loc[MI_train['MonthlyIncome'].isnull(), 'MonthlyIncome']=rfr.predict(X_unknown).round(0)

In [ ]:
#随机森林法填补NumberOfDependents
ND_train=train.copy()
ND_known=ND_train[ND_train['NumberOfDependents'].notnull()]
ND_unknown=ND_train[ND_train['NumberOfDependents'].isnull()]
# 删除MI_know中的NumberOfDependents列，但不改变原有的MI_know中的数据，而是返回另一个dataframe：X_known来存放删除后的数据
X_known=ND_known.drop('NumberOfDependents',axis=1)
# y_known是一个series
y_known=ND_known['NumberOfDependents']
# 删除ND_unknown中的NumberOfDependents列，但不改变原有的ND_unknown中的数据，而是返回另一个dataframe：X_unknown来存放删除后的数据
X_unknown=ND_unknown.drop('NumberOfDependents',axis=1)
# 使用scikit-learn中的RandomForestRegressor
rfr=RandomForestRegressor(random_state=0, n_estimators=100, max_depth=3, n_jobs=-1)
rfr.fit(X_known, y_known)
train.loc[ND_train['NumberOfDependents'].isnull(), 'NumberOfDependents']=rfr.predict(X_unknown).round(0)

2.2查看有无异常值

In [ ]:
train.describe()

In [ ]:
test.describe()

可以近似认为无异常

3.搭建三层神经网络

3.1倒入神经网络需要的库

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#搭建网络
model = Sequential(
    [Dense(35, input_dim=10, activation="relu"),
    Dense(1, activation="sigmoid")]
)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
#训练
X = np.delete(train.values, 0, axis=1)
model.fit(X, train[:]['SeriousDlqin2yrs'], epochs=20, verbose=1) 

In [ ]:
x_test = test.drop(['SeriousDlqin2yrs'],axis=1).values

In [ ]:
predictions = model.predict(x_test)[:,0]
StackingSubmission = pd.DataFrame({'Id':Unnamed, 'Probability':predictions})
StackingSubmission.to_csv("X.csv", index=False)
StackingSubmission.head()

4.发现有缺失值

In [ ]:
#用中位数填充
StackingSubmission['Probability'].fillna(StackingSubmission['Probability'].median(),inplace=True)
StackingSubmission.info()
StackingSubmission.to_csv("X.csv", index=False)